In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import json
import pandas as pd
from nltk import word_tokenize
import gensim 
from gensim.models import Word2Vec
import re
import csv
import sys
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
import spacy
import en_core_web_sm
import nltk
nltk.download('vader_lexicon')

unable to import 'smart_open.gcs', disabling that module
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\shubh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
#Merging subfolder of a particular year. Run this until all files of a year are merged into final output file(output_2018)
with open('output_2018_1.txt',encoding="utf-8") as fp: 
    data = fp.read() 
    
with open('output_2018.txt',encoding="utf-8") as fp: 
    data2 = fp.read() 
    
data += "\n"
data += data2 

with open ('output_2018.txt', 'w',encoding="utf-8") as fp: 
    fp.write(data) 

In [4]:
data_df = open("output_2018.txt", "r",encoding="utf-8").readlines()
for i in range(len(data_df)):
    data_df[i] = str(data_df[i]).replace('[','').replace(']','')

In [5]:
nlp = en_core_web_sm.load()
matcher = PhraseMatcher(nlp.vocab)

In [6]:
#List of names of person murdered in 2018. Created after running "homicide articles by name"
name_list_df = pd.read_csv('Name_List_2018.csv',encoding="utf-8")  

In [8]:
#Filtering all people of white community. Same code can be used for black community after changing race values to black.
name_list=[]
for i in range(len(name_list_df)):
    if ((name_list_df['Race'][i] in ('W', 'W H','W N','WH','H')) and name_list_df['Number of Articles'][i]!= 0)  :
        name_list.append(name_list_df['Full Name'][i])

In [9]:
print(name_list)

['John Colon']


In [10]:
#Finding the articles where these names are mentioned in the homicide subset dataset. Storing articles in data_df_name.
count = 0
count_F=[]
data_df_name =[]
for i in range(len(name_list)):
    matcher.add("Phrase Matching", None, nlp(name_list[i]))
    print(nlp(name_list[i]))
    for j in range(len(data_df)):
        if(count < articles_count[i]):
            doc = nlp(data_df[j])
            matches = matcher(doc)
            #print(len(matches))
            if (len(matches) > 0):
                print(matches)
                count= count + 1
                data_df_name.append(data_df[j])
        else:
            break
    matcher.remove("Phrase Matching")
    count_F.append(count)
    count=0

John Colon
[(11356100181062323261, 23, 25)]


In [ ]:
#Writing the articles mentioning name explicitly in a txt file. 
#This code is required if you want to merge all the data of specific community of complete 5 years.
#and then do topic modelling of complete 5 years together. To do that you will run above code cells again for all the years.
#and create separate txt for every year specific to community and then once 5 files are created, merge them and 
#Then run below topic modeling code. If you don't want to merge all years skip this and next cell. 
#Start with Topic modelling of particular year.

filename = 'homicide_white_data_2018.txt'
resultFyle = open(filename,'w',encoding="utf-8")
for r in data_df_name:
    resultFyle.write(r)
resultFyle.close()

In [ ]:
#Code to write txt content created after merging all files into one txt file of 5 years into data frame.
data_df_name = open("homicide_white_complete_data.txt", "r",encoding="utf-8").readlines()
for i in range(len(data_df)):
    data_df[i] = str(data_df_name[i]).replace('[','').replace(']','')
    data_df[i] = str(data_df_name[i]).replace('\'','')

# Topic modelling

In [11]:
data = list(dict.fromkeys(data_df_name))

In [12]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come','said','tell'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [13]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  
    
data_words = list(sent_to_words(data))


In [14]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
# or do
# !conda install -c conda-forge spacy-model-en_core_web_md 
# and use nlp=spacy.load('en_core_web_sm') instead in below function.
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)

In [27]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

print(lda_model.print_topics())

[(0, '0.034*"colon" + 0.034*"police" + 0.034*"tip" + 0.033*"resident" + 0.033*"die" + 0.033*"identify" + 0.033*"local" + 0.033*"incident" + 0.033*"wish" + 0.033*"call"'), (1, '0.052*"tip" + 0.052*"police" + 0.052*"colon" + 0.031*"gunshot" + 0.031*"anonymously" + 0.031*"multiple" + 0.031*"wound" + 0.031*"crimestopper" + 0.031*"year" + 0.031*"statement"')]


In [ ]:
#Only works when number of topics are 4 in above cell(num_topics)

from collections import Counter
import matplotlib.colors as mcolors
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 7500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=10)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count for White Community', fontsize=22, y=1.05)    
plt.show()

In [29]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis
#pyLDAvis.save_html(vis, 'output_2018_white.html')